In [1]:
from __future__ import unicode_literals, print_function, division

# import basic lib
import re
import math
import random
import string
import unicodedata
from io import open

# import pytorch
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch import optim
from torch.autograd import Variable
from torch.nn.utils import clip_grad_norm_

# import loss func
import masked_cross_entropy

# check device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
PAD_idx = 0
SOS_idx = 1
EOS_idx = 2
UNK_idx = 3

USE_CUDA = True

class Preprocessor:
    '''
    class for preprocessing
    '''
    def __init__(self, name):
        '''
        initialize vocab and counter
        '''
        self.name = name
        self.w2idx = {}
        self.counter = {}
        self.idx2w = {0: "<sos>", 1: "<eos>", 2:"<unk>"}
        self.num = 2

    def SentenceAdder(self, sentence):
        '''
        Add a sentence to dataset
        '''
        for word in sentence.split(' '):
            self.WordAdder(word)

    def WordAdder(self, word):
        '''
        Add single word to dataset and update vocab and counter
        '''
        if word in self.w2idx:
            self.counter[word] += 1
        else:
            self.w2idx[word] = self.num
            self.counter[word] = 1
            self.idx2w[self.num] = word
            self.num += 1
            
    def trim(self, min_count=2):
        '''
        Trim to remove non-frequent word
        '''
        keep = []
        for k, v in self.counter.items():
            if v >= min_count: keep.append(k)
        print(self.name+':')
        print('Total words', len(self.w2idx))
        print('After Trimming', len(keep))
        print('Keep Ratio %', len(keep) / len(self.w2idx))
        self.w2idx = {}
        self.counter = {}
        self.idx2w = {0: "<sos>", 1: "<eos>", 2:"<unk>"}
        self.num = 2
        for w in keep:
            self.WordAdder(w)

In [3]:
def Uni2Ascii(s):
    '''
    transfer from unicode to ascii
    '''
    return ''.join(c for c in unicodedata.normalize('NFD', s)
                    if unicodedata.category(c) != 'Mn')

def StrCleaner(s):
    '''
    trim, delete non-letter and lowercase string
    '''
    s = Uni2Ascii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

def DataReader(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[StrCleaner(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Preprocessor(lang2)
        output_lang = Preprocessor(lang1)
    else:
        input_lang = Preprocessor(lang1)
        output_lang = Preprocessor(lang2)

    return input_lang, output_lang, pairs

In [4]:
MIN_LENGTH = 5
MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)

def filterPair(p):
    return MIN_LENGTH <= len(p[0].split(' ')) <= MAX_LENGTH and \
        MIN_LENGTH <= len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [5]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = DataReader(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.SentenceAdder(pair[0])
        output_lang.SentenceAdder(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.num)
    print(output_lang.name, output_lang.num)
    return input_lang, output_lang, pairs


src, tgt, pairs = prepareData('eng', 'fra', True)
src.trim()
tgt.trim()
print(random.choice(pairs))

Reading lines...
Read 135842 sentence pairs
Trimmed to 9184 sentence pairs
Counting words...
Counted words:
fra 3978
eng 2732
fra:
Total words 3976
After Trimming 2236
Keep Ratio % 0.562374245472837
eng:
Total words 2730
After Trimming 1718
Keep Ratio % 0.6293040293040293
['je suis un excellent joueur d echecs .', 'i am a very good chess player .']


In [6]:
def sentence2idx(preprocessor, sentence):
    '''
    Read sentence and translate into word index plus eos
    '''
    return [SOS_idx] + [preprocessor.w2idx[w] if w in preprocessor.w2idx \
            else UNK_idx for w in sentence.split(' ')] + [EOS_idx]

def pad(seq, max_len):
    '''
    Add padding to sentence with different length
    '''
    seq += [PAD_idx for i in range(max_len - len(seq))]
    return seq

def random_batch(src, tgt, batch_size=5):
    '''
    Randomly generate batch data
    '''
    inputs, target = [], []
    
    # Choose batch randomly
    for _ in range(batch_size):
        pair = random.choice(pairs)
        inputs.append(sentence2idx(src, pair[0]))
        target.append(sentence2idx(tgt, pair[1]))
        
    # Sort by length
    seq_pairs = sorted(zip(inputs, target), key=lambda p: len(p[0]), reverse=True)
    inputs, target = zip(*seq_pairs)
    
    # Obtain length of each sentence and pad
    input_lens = [len(s) for s in inputs]
    input_max = max(input_lens)
    input_padded = [pad(s, input_max) for s in inputs]
    target_lens = [len(s) for s in target]
    target_max = max(target_lens)
    target_padded = [pad(s, target_max) for s in target]

    # Create Variable
    input_vars = Variable(torch.LongTensor(input_padded)).transpose(0, 1)
    input_lens = Variable(torch.LongTensor(input_lens))
    target_vars = Variable(torch.LongTensor(target_padded)).transpose(0, 1)
    target_lens = Variable(torch.LongTensor(target_lens))

    if USE_CUDA:
        input_vars = input_vars.cuda()
        input_lens = input_lens.cuda()
        target_vars = target_vars.cuda()
        target_lens = target_lens.cuda()

    return input_vars, input_lens, target_vars, target_lens

In [7]:
class Encoder(nn.Module):
    '''
    Define encoder and forward process
    '''
    def __init__(self, dim_input, dim_embed, dim_hidden, num_layers, dropout):
        super(Encoder, self).__init__()
        self.dim_input = dim_input
        self.dim_hidden = dim_hidden
        self.dim_embed = dim_embed
        self.embed = nn.Embedding(dim_input, dim_embed)
        self.cell = nn.GRU(dim_embed, dim_hidden, 
                          num_layers, dropout=dropout, 
                          bidirectional=True)
        
    def forward(self, inputs, inputs_lens, hidden=None):
        '''
        We need to sum the outputs since bi-diretional is used
        '''
        embedded = self.embed(inputs)
        packed = nn.utils.rnn.pack_padded_sequence(embedded, inputs_lens)
        outputs, hidden = self.cell(packed, hidden)
        outputs, output_lengths = nn.utils.rnn.pad_packed_sequence(outputs)
        outputs = outputs[:, :, :self.dim_hidden] + \
                    outputs[:, :, self.dim_hidden:]
        return outputs, hidden


class Attention(nn.Module):
    '''
    Define attention mechanism
    '''
    def __init__(self, dim_hidden):
        super(Attention, self).__init__()
        self.dim_hidden = dim_hidden
        # 2*dim_hidden is needed since bi-direction is used
        self.attn = nn.Linear(2*self.dim_hidden, dim_hidden)
        self.v = nn.Parameter(torch.rand(dim_hidden))
        stdv = 1. / math.sqrt(self.v.size(0))
        self.v.data.uniform_(-stdv, stdv)

    def forward(self, hidden, encoder_outputs):
        timestep = encoder_outputs.size(0)
        h = hidden.repeat(timestep, 1, 1).transpose(0, 1)
        encoder_outputs = encoder_outputs.transpose(0, 1)
        scores = self.score(h, encoder_outputs)
        return F.relu(scores).unsqueeze(1)

    def score(self, hidden, encoder_outputs):
        e = F.softmax(self.attn(torch.cat([hidden, encoder_outputs], 2)),dim=1)
        e = e.transpose(1, 2)
        v = self.v.repeat(encoder_outputs.size(0), 1).unsqueeze(1)
        e = torch.bmm(v, e)
        return e.squeeze(1)


class Decoder(nn.Module):
    '''
    Define decoder with attention
    '''
    def __init__(self, dim_embed, dim_hidden, dim_output, num_layers, dropout):
        super(Decoder, self).__init__()
        self.dim_embed = dim_embed
        self.dim_hidden = dim_hidden
        self.dim_output = dim_output
        self.num_layers = num_layers

        self.embed = nn.Embedding(dim_output, dim_embed)
        self.dropout = nn.Dropout(dropout, inplace=True)
        self.attention = Attention(dim_hidden)
        self.cell = nn.GRU(dim_hidden + dim_embed, dim_hidden,
                          num_layers, dropout=dropout)
        self.out = nn.Linear(2*dim_hidden, dim_output)

    def forward(self, inputs, last_hidden, encoder_outputs):
        # Get the embedding of the current input word (last output word)
        embedded = self.embed(inputs).unsqueeze(0)  # (1,B,N)
        embedded = self.dropout(embedded)
        # Calculate attention weights and apply to encoder outputs
        attn_weights = self.attention(last_hidden[-1], encoder_outputs)
        context = attn_weights.bmm(encoder_outputs.transpose(0, 1))  # (B,1,N)
        context = context.transpose(0, 1)  # (1,B,N)
        # Combine embedded input word and attended context, run through RNN
        rnn_input = torch.cat([embedded, context], 2)
        #print(embedded.size())
        #print(context.size())
        #print(rnn_input.size())
        #print(last_hidden.size())
        output, hidden = self.cell(rnn_input, last_hidden)
        output = output.squeeze(0)  # (1,B,N) -> (B,N)
        context = context.squeeze(0)
        # For Debug
        #print(output.size())
        #print(context.size())
        torch.cat([output, context], 1)
        output = self.out(torch.cat([output, context], 1))
        output = F.log_softmax(output, dim=1)
        return output, hidden, attn_weights


class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, src, src_len, tgt, tgt_len, teacher_forcing_ratio=0.5):
        batch_size = src.size(1)
        max_len = tgt.size(0)
        vocab_size = self.decoder.dim_output
        outputs = Variable(torch.zeros(max_len, batch_size, vocab_size)).cuda()

        encoder_output, hidden = self.encoder(src, src_len)
        hidden = hidden[:self.decoder.num_layers]
        output = Variable(tgt.data[0, :])
        for t in range(1, max_len):
            output, hidden, attn_weights = self.decoder(
                    output, hidden, encoder_output)
            outputs[t] = output
            # Randomly choose whether to use teacher force or not
            is_teacher = random.random() < teacher_forcing_ratio
            top1 = output.data.max(1)[1]
            output = Variable(tgt.data[t] if is_teacher else top1).cuda()
        return outputs

In [8]:
batch_size = 2
hidden_size = 8
embed_size = 8
n_layers = 3
encoder_test = Encoder(src.num, embed_size, hidden_size, n_layers, dropout=0.5)
decoder_test = Decoder(embed_size, hidden_size, tgt.num, n_layers, dropout=0.5)

In [9]:
net = Seq2Seq(encoder_test,decoder_test).cuda()
opt = optim.Adam(net.parameters(),lr=0.01)
print(net)

Seq2Seq(
  (encoder): Encoder(
    (embed): Embedding(2238, 8)
    (cell): GRU(8, 8, num_layers=3, dropout=0.5, bidirectional=True)
  )
  (decoder): Decoder(
    (embed): Embedding(1720, 8)
    (dropout): Dropout(p=0.5, inplace)
    (attention): Attention(
      (attn): Linear(in_features=16, out_features=8, bias=True)
    )
    (cell): GRU(16, 8, num_layers=3, dropout=0.5)
    (out): Linear(in_features=16, out_features=1720, bias=True)
  )
)


In [10]:
grad_clip = 10

for step in range(1,100):
    total_loss = 0
    input_batches, input_lengths,\
        target_batches, target_lengths = random_batch(src,tgt,batch_size)
    opt.zero_grad()
    output = net(input_batches, input_lengths, target_batches, target_lengths)
    
    # For Debug
    #print('target lengths', target_lengths)
    
    loss = masked_cross_entropy.compute_loss(
        output.transpose(0, 1).contiguous(),
        target_batches.transpose(0, 1).contiguous(),
        target_lengths
    )
    print('loss = ', loss.item())
    
  
    
    clip_grad_norm_(net.parameters(), grad_clip)
    loss.backward()
    opt.step()

loss =  7.468875408172607
loss =  7.464125633239746
loss =  7.357632160186768
loss =  7.275870323181152
loss =  7.153682708740234
loss =  7.098086357116699
loss =  6.893886566162109
loss =  6.961111068725586
loss =  6.725880146026611
loss =  6.515769004821777
loss =  6.1591291427612305
loss =  5.801735877990723
loss =  5.928849220275879
loss =  5.742967128753662
loss =  6.009059429168701
loss =  5.2586350440979
loss =  5.6672492027282715
loss =  5.482161521911621
loss =  5.435443878173828
loss =  6.089744567871094
loss =  5.082825183868408
loss =  4.550608158111572
loss =  5.751797199249268
loss =  5.643023490905762
loss =  5.424783706665039
loss =  5.091669082641602
loss =  4.999526500701904
loss =  5.504539966583252
loss =  4.679466724395752
loss =  5.477797031402588
loss =  5.193601131439209
loss =  5.199742317199707
loss =  5.492315292358398
loss =  5.242008686065674
loss =  5.068694114685059
loss =  5.543230056762695
loss =  5.291325569152832
loss =  5.558704853057861
loss =  5.07